## Spectrum Simulation Example

In this example, we will simulate the NMR spectrum of a Pyridine molecule.

- **Molecule and Spins**: The simulation incorporates all NMR active nuclei of the Pyridine molecule.

- **Interactions Included**:
    - **Coherent Interactions**:
        - Zeeman interaction.
        - Chemical shift.
        - Scalar $J$-coupling.
    - **Incoherent Interactions**:
        - Dipolar coupling.
        - Shielding.
        - Quadrupolar interaction.
        - Scalar relaxation of the second kind.

This example provides an overview of how to simulate NMR spectra using Spinguin using its user-friendly API.

### Imports

To begin, we import all the necessary modules.

- **NumPy**: Used for creating arrays required for input parameters and storing results.
- **Matplotlib**: Utilized for plotting the simulation results.
- **Spinguin**: Provides specialized functionality for simulating NMR experiments.

In [ ]:
# Import necessary modules:
import numpy as np
import matplotlib.pyplot as plt
import spinguin as sg

### Simulation Parameters

Next, we define the simulation parameters.

The following experimental conditions are defined:

- **Magnetic Field (T)**: The strength of the static magnetic field $B_0$.
- **Temperature (K)**: The temperature at which the simulation is conducted.

The following acquisition parameters are defined:

- **Isotope**: Defines the isotope that is to be measured.
- **Dwell Time (s)**: Defines the sampling frequency. Must be set such that the Nyquist criterion is met.
- **Number of Points**: Defines the number of points in the FID. Together with dwell time, determines the total acquisition time.
- **Angle**: Defines the pulse angle.
- **Center Frequencies (ppm)**: Sets the offset of the spectrometer to the specified frequencies. This should be set close to the center of the spectrum.

In [ ]:
# Experimental conditions
sg.parameters.magnetic_field = 9.4
sg.parameters.temperature = 295

# Acquisition parameters:
isotope = "1H"
dwell_time = sg.spectral_width_to_dwell_time(2, isotope)
npoints = 12500
angle = 90
center_frequency = 8

### Define the Spin System

Next, we perform the following:

- **Initialize Spin System**: We create a spin system that consists of 5 protons (1H) and 1 nitrogen (14N).
- **Define Maximum Spin Order**: To speed up the calculations, the maximum spin order is restricted to include at most 3 spins.
- **Build the Basis**: We build the Liouville space basis set for the spin system.

In [ ]:
# Define the spin system
spin_system = sg.spin_system(['1H', '1H', '1H', '1H', '1H', '14N'])
spin_system.basis.max_spin_order = 3
spin_system.basis.build()

### Define the Spin System Parameters

We define the following parameters:

- **Chemical Shifts (ppm)**: The chemical shifts for each nucleus.
- **J Couplings (Hz)**: The $J$-coupling constants between nuclei.
- **XYZ Coordinates (Å)**: The spatial positions of the nuclei in Cartesian coordinates.
- **Shielding Tensors (ppm)**: The nuclear shielding tensors in Cartesian coordinates.
- **EFG Tensors (a.u.)**: The electric field gradient tensors in Cartesian coordinates.

In [ ]:
# Set the spin system properties
spin_system.chemical_shifts = np.array([8.56, 8.56, 7.47, 7.47, 7.88, 95.94])

spin_system.J_couplings = np.array([
    [ 0,     0,      0,      0,      0,      0],
    [-1.04,  0,      0,      0,      0,      0],
    [ 4.85,  1.05,   0,      0,      0,      0],
    [ 1.05,  4.85,   0.71,   0,      0,      0],
    [ 1.24,  1.24,   7.55,   7.55,   0,      0],
    [ 8.16,  8.16,   0.87,   0.87,  -0.19,   0]
])

spin_system.xyz = np.array([
    [ 2.0495335,  0.0000000, -1.4916842],
    [-2.0495335,  0.0000000, -1.4916842],
    [ 2.1458878,  0.0000000,  0.9846086],
    [-2.1458878,  0.0000000,  0.9846086],
    [ 0.0000000,  0.0000000,  2.2681296],
    [ 0.0000000,  0.0000000, -1.5987077]
])

spin_system.shielding = np.zeros((6, 3, 3))
spin_system.shielding[5] = np.array([
    [-406.20,  0.00,    0.00],
    [   0.00, 299.44,   0.00],
    [   0.00,   0.00, -181.07]
])

spin_system.efg = np.zeros((6, 3, 3))
spin_system.efg[5] = np.array([
    [ 0.3069,  0.0000,  0.0000],
    [ 0.0000,  0.7969,  0.0000],
    [ 0.0000,  0.0000, -1.1037]
])

### Define the relaxation theory

We will treat the relaxation using Redfield theory. The following parameters are set:

- **Theory**: We specify that Redfield theory is used.
- **SR2K**: We specify that scalar relaxation of the second kind is considered.
- **Thermalization**: Thermalization of the relaxation superoperator (Levitt-di Bari) is enabled.
- **Rotational Correlation Time (ps)**: The characteristic time for molecular tumbling, affecting relaxation processes.

In [ ]:
# Set the relaxation theory
spin_system.relaxation.theory = "redfield"
spin_system.relaxation.sr2k = True
spin_system.relaxation.thermalization = True
spin_system.relaxation.tau_c = 50e-12

### Perform the pulse-and-acquire experiment

To simulate the spectrum, we perform the pulse-and-acquire experiment, which returns the free induction decay (FID) signal.

In [ ]:
# Acquire
fid = sg.pulse_and_acquire(spin_system, isotope, center_frequency, npoints, dwell_time, angle)

### Perform the Fourier Transform to Obtain the Spectrum

- **Fourier Transform**: We then take the Fourier transform of the magnetization data to get the NMR spectrum.
- **Consider the Rotating Frame Frequency**: The detection was performed in a rotating frame that involved the high-frequency component arising from bare-nucleus Zeeman interaction and the user-specified center frequency. The rotating frame frequency is added back to the detected frequencies.
- **Frequency to Chemical Shift Conversion**: The frequency axis is converted to chemical shifts in ppm for better interpretability of the spectrum.

In [ ]:
# Compute the real part of the spectrum.
freqs, spec = sg.spectrum(fid, dwell_time)

# Convert rotating frame frequencies back to labframe
rotframe_frequency = sg.resonance_frequency(isotope, center_frequency)
freqs = freqs + rotframe_frequency

# Convert frequencies to ppm
spectrometer_frequency = sg.resonance_frequency(isotope)
chemical_shifts = sg.frequency_to_chemical_shift(freqs, spectrometer_frequency, spectrometer_frequency)

### Plot the Results

Congratulations! The simulation is complete. We will then plot the results.

In [ ]:
# Plot the magnetization over time
t_axis = sg.time_axis(npoints, dwell_time)
plt.figure(figsize=(8, 4))
plt.plot(t_axis, np.real(fid), color='blue')
plt.title('Transverse Magnetization')
plt.xlabel('Time (s)')
plt.ylabel('Magnetization')
plt.grid()
plt.xlim(0, npoints*dwell_time)
plt.tight_layout()
plt.show()

In [ ]:
# Create a figure with subplots
fig, axes = plt.subplots(4, 1, figsize=(10, 16), gridspec_kw={'height_ratios': [1, 1, 1, 1]})

# Plot the full spectrum on the top
axes[0].plot(chemical_shifts, spec, color='blue')
axes[0].set_title('Full Spectrum')
axes[0].set_xlabel('Chemical Shift (ppm)')
axes[0].set_ylabel('Intensity')
axes[0].grid()
axes[0].set_xlim(8.8, 7.2)

# Plot the zoomed-in spectrum (left) in the second subplot
axes[1].plot(chemical_shifts, spec, color='blue')
axes[1].set_title('Spectrum (Zoomed In - Left)')
axes[1].set_xlabel('Chemical Shift (ppm)')
axes[1].set_ylabel('Intensity')
axes[1].grid()
axes[1].set_xlim(7.49, 7.45)

# Plot the zoomed-in spectrum (middle) in the third subplot
axes[2].plot(chemical_shifts, spec, color='blue')
axes[2].set_title('Spectrum (Zoomed In - Middle)')
axes[2].set_xlabel('Chemical Shift (ppm)')
axes[2].set_ylabel('Intensity')
axes[2].grid()
axes[2].set_xlim(7.91, 7.85)

# Plot the zoomed-in spectrum (right) at the bottom
axes[3].plot(chemical_shifts, spec, color='blue')
axes[3].set_title('Spectrum (Zoomed In - Right)')
axes[3].set_xlabel('Chemical Shift (ppm)')
axes[3].set_ylabel('Intensity')
axes[3].grid()
axes[3].set_xlim(8.57, 8.55)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()